In [ ]:
#! pip install pytorch-tabnet

In [1]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.model_selection import cross_validate, train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

random_state = 101

In [ ]:
path_import_and_export = "../../../../Thesis_data/processed_data/"

X_train = pd.read_csv(path_import_and_export + "ontime_reporting_X_train.csv")
y_train = pd.read_csv(path_import_and_export + "ontime_reporting_y_train.csv")
X_test = pd.read_csv(path_import_and_export + "ontime_reporting_X_test.csv")
y_test = pd.read_csv(path_import_and_export + "ontime_reporting_y_test.csv")

In [ ]:
# np.ravel(y_train)
# np.ravel(y_test)

In [ ]:
# np.array(X_train)
# np.array(y_train)
# np.array(X_test)
# np.array(y_test)

In [ ]:
# np.array(X_train)
# y_train.to_numpy()
# X_test.to_numpy()
# y_test.to_numpy()

In [5]:

#https://pypi.org/project/pytorch-tabnet/

path_import_and_export = "../../../../Thesis_data/processed_data/"

X_train = pd.read_csv(path_import_and_export + "ontime_reporting_X_train.csv").values.astype(float)
y_train = pd.read_csv(path_import_and_export + "ontime_reporting_y_train.csv").values
X_test = pd.read_csv(path_import_and_export + "ontime_reporting_X_test.csv").values.astype(float)
y_test = pd.read_csv(path_import_and_export + "ontime_reporting_y_test.csv").values

y_train = np.ravel(y_train)
y_test = np.ravel(y_test)

print(X_train)

def objective_tabnet(trial):

    params = {
        "n_d": trial.suggest_int("n_d", 8,  64, step=4),
        "n_steps": trial.suggest_int("n_steps", 3, 10),
        "gamma": trial.suggest_float("gamma", 1.0, 2.0, step=0.1),
        "n_shared": trial.suggest_int("n_shared", 1, 5),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 1e-3, 3e-3, log=True),
        "mask_type": trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    }

    model_tabnetClass = TabNetClassifier(**params, seed=random_state, verbose=2, optimizer_params=dict(lr=2e-2, weight_decay=1e-5), n_a=params["n_d"]) #device_name="cuda"

    skfold = StratifiedKFold(n_splits=2, random_state=random_state, shuffle=True)
    auc_scores = []

    for train_index, val_index in skfold.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        model_tabnetClass.fit(X_train = X_train_fold,
                              y_train = y_train_fold,
                              eval_set = [(X_val_fold, y_val_fold)],
                              patience=5,
                              max_epochs=1,
                              batch_size=1500,
                              eval_metric=["auc"]
                              )

        y_pred = model_tabnetClass.predict(X_val_fold)

        auc_scores.append(roc_auc_score(y_val_fold, y_pred))

    return np.mean(auc_scores)


optuna.logging.set_verbosity(optuna.logging.DEBUG)
study = optuna.create_study(direction="maximize", study_name='TabNet')
study.optimize(objective_tabnet, n_trials=1)

[I 2023-10-06 16:05:32,790] A new study created in memory with name: TabNet
C:\Users\Rvanl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pytorch_tabnet\abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.42109 | val_0_auc: 0.7348  |  0:17:43s
Stop training because you reached max_epochs = 1 with best_epoch = 0 and best_val_0_auc = 0.7348


C:\Users\Rvanl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)
[W 2023-10-06 17:36:56,426] Trial 0 failed with parameters: {'n_d': 8, 'n_steps': 7, 'gamma': 1.6, 'n_shared': 3, 'lambda_sparse': 0.001124255306326657, 'mask_type': 'sparsemax'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\Rvanl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\Rvanl\AppData\Local\Temp\ipykernel_5540\1096735439.py", line 33, in objective_tabnet
    model_tabnetClass.fit(X_train = X_train_fold,
  File "C:\Users\Rvanl\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qb

KeyboardInterrupt: 

In [ ]:
path_import_and_export = "../../../../Thesis_data/processed_data/"

X_train = pd.read_csv(path_import_and_export + "ontime_reporting_X_train.csv")
y_train = pd.read_csv(path_import_and_export + "ontime_reporting_y_train.csv", squeeze=True)
X_test = pd.read_csv(path_import_and_export + "ontime_reporting_X_test.csv")
y_test = pd.read_csv(path_import_and_export + "ontime_reporting_y_test.csv", squeeze=True)

def objective_tabnet(trial):
    params = {
        "n_d": trial.suggest_int("n_d", 8, 16, step=4),
        "n_a": trial.suggest_int("n_a", 8, 16, step=4),
        "n_steps": trial.suggest_int("n_steps", 3, 10),
        "cat_emb_dim": trial.suggest_int("cat_emb_dim", 1, 5),
        "gamma": trial.suggest_float("subsample", 1.0, 2.0, step=0.01),
        "n_shared": trial.suggest_int("colsample_bytree", 1, 5),
        "lambda_sparse": trial.suggest_float("lambda_sparse", 1e-3, 3e-3, log=True),
        "mask_type": trial.suggest_categorical("mask_type", ["entmax", "sparsemax"])
    }

    model_tabnetClass = TabNetClassifier(**params, seed=random_state, verbose=2)

    skfold = StratifiedKFold(n_splits=3, random_state=random_state, shuffle=True)
    auc_scores = []

    for train_index, val_index in skfold.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
        y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

        model_tabnetClass.fit(X_train = X_train_fold.values,
                              y_train = y_train_fold.values,
                              eval_set = [(X_val_fold.values, y_val_fold.values)],
                              patience=5,
                              max_epochs=1,
                              batch_size=1500,
                              eval_metric=["auc"]
                              )

        y_pred = model_tabnetClass.predict(X_val_fold.values)#[:, 1]
        auc_scores.append(roc_auc_score(y_val_fold.values, y_pred))

    return np.mean(auc_scores)


optuna.logging.set_verbosity(optuna.logging.DEBUG)
study = optuna.create_study(direction="maximize", study_name='TabNet')
study.optimize(objective_tabnet, n_trials=1)

In [ ]:
def objective_tabnet(trial):
    params = {
        "n_d": trial.suggest_int("n_d", 8, 64, step=8),
        "n_a": trial.suggest_int("n_a", 8, 64, step=8),
        "n_steps": trial.suggest_int("n_steps", 3, 10),
        "gamma": trial.suggest_float("subsample", 1.0, 2.0, step=0.01),
        "n_shared": trial.suggest_int("colsample_bytree", 1, 5)
    }

    model_tabnetClass = TabNetClassifier(**params, seed=random_state, n_jobs=-1, verbose=1)
    
    skfold = StratifiedKFold(n_splits=3, random_state=random_state, shuffle=True)
    auc_scores = []

    for train_index, val_index in skfold.split(X_train, y_train):
        X_train_fold, y_train_fold = X_train.iloc[train_index], y_train.iloc[train_index]
        X_val_fold, y_val_fold = X_train.iloc[val_index], y_train.iloc[val_index]

        model_tabnetClass.fit(X_train_fold, y_train_fold,
                              eval_set = [(X_val_fold, y_val_fold)],
                              patience=5,
                              max_epochs=10,
                              eval_metric=["auc"])
        
        y_pred = model_tabnetClass.predict(X_val_fold)#[:, 1]
        auc_scores.append(roc_auc_score(y_val_fold, y_pred))

    return np.mean(auc_scores)

In [ ]:
optuna.logging.set_verbosity(optuna.logging.DEBUG)
study = optuna.create_study(direction="maximize", study_name='TabNet')
study.optimize(objective_tabnet, n_trials=1)

In [4]:
TabNet_params

{'n_d': 8,
 'n_a': 16,
 'n_steps': 4,
 'cat_emb_dim': 2,
 'subsample': 1.94,
 'colsample_bytree': 3,
 'lambda_sparse': 0.0024674522291325534,
 'mask_type': 'sparsemax'}

In [3]:
TabNet_params = study.best_params
final_tabnet_model = TabNetClassifier(**TabNet_params, seed=random_state, verbose=1)
final_tabnet_model.fit(X_train, y_train)

TypeError: TabModel.__init__() got an unexpected keyword argument 'subsample'

In [ ]:
y_pred = final_tabnet_model.predict(X_test)
test_auroc = roc_auc_score(y_test, y_pred)

In [ ]:
print(test_auroc)